# Part II: LoRA Fine-tuning Using NeMo Customizer

This notebook covers the following:

0. [Prerequisites: Configurations, Health Checks, and Namespaces](#step-0)
1. [Upload Data to NeMo Datastore](#step-1)
2. [LoRA Customization with NeMo Customizer](#step-2)
3. [Running Inference on the Customized Model with NVIDIA NIM](#step-3)

In [1]:
import os
import json
import random
import requests
from openai import OpenAI
from nemo_microservices import NeMoMicroservices

<a id="step-0"></a>
## Prerequisites: Configurations, Health Checks, and Namespaces

Before you proceed, make sure that you completed the first notebook on data preparation to obtain the assets required to follow along.

### Configure NeMo Microservices Endpoints

This section includes importing required libraries, configuring endpoints, and performing health checks to ensure that the NeMo Data Store, NIM, and other services are running correctly.

In [2]:
from config import *

# Initialize NeMo Microservices SDK client
nemo_client = NeMoMicroservices(
    base_url=NEMO_URL,
    inference_base_url=NIM_URL,
)

In [3]:
print(f"Data Store endpoint: {NDS_URL}")
print(f"Entity Store, Customizer, Evaluator endpoint: {NEMO_URL}")
print(f"NIM endpoint: {NIM_URL}")
print(f"Namespace: {NMS_NAMESPACE}")
print(f"Base Model for Customization: {BASE_MODEL}@{BASE_MODEL_VERSION}")

Data Store endpoint: http://data-store.test
Entity Store, Customizer, Evaluator endpoint: http://nemo.test
NIM endpoint: http://nim.test
Namespace: xlam-tutorial-ns
Base Model for Customization: meta/llama-3.2-1b-instruct@v1.0.0+A100


### Configure Path to Prepared data

The following code sets the paths to the prepared dataset files.

In [4]:
# Path where data preparation notebook saved finetuning and evaluation data
DATA_ROOT = os.path.join(os.getcwd(), "data")
CUSTOMIZATION_DATA_ROOT = os.path.join(DATA_ROOT, "customization")
VALIDATION_DATA_ROOT = os.path.join(DATA_ROOT, "validation")
EVALUATION_DATA_ROOT = os.path.join(DATA_ROOT, "evaluation")

# Sanity checks
train_fp = f"{CUSTOMIZATION_DATA_ROOT}/training.jsonl"
assert os.path.exists(train_fp), f"The training data at '{train_fp}' does not exist. Please ensure that the data was prepared successfully."

val_fp = f"{VALIDATION_DATA_ROOT}/validation.jsonl"
assert os.path.exists(val_fp), f"The validation data at '{val_fp}' does not exist. Please ensure that the data was prepared successfully."

test_fp = f"{EVALUATION_DATA_ROOT}/xlam-test-single.jsonl"
assert os.path.exists(test_fp), f"The test data at '{test_fp}' does not exist. Please ensure that the data was prepared successfully."

### Resource Organization Using Namespace

You can use a [namespace](https://docs.nvidia.com/nemo/microservices/latest/manage-entities/namespaces/index.html) to isolate and organize the artifacts in this tutorial.

#### Create Namespace

Both Data Store and Entity Store use namespaces. The following code creates namespaces for the tutorial.

In [5]:
def create_namespaces(nemo_client, ds_host, namespace):
    # Create namespace in Entity Store
    try:
        namespace_obj = nemo_client.namespaces.create(id=namespace)
        print(f"Created namespace in Entity Store: {namespace_obj.id}")
    except Exception as e:
        # Handle if namespace already exists
        if "409" in str(e) or "422" in str(e):
            print(f"Namespace {namespace} already exists in Entity Store")
        else:
            raise e

    # Create namespace in Data Store (still using requests as SDK doesn't cover Data Store)
    nds_url = f"{ds_host}/v1/datastore/namespaces"
    resp = requests.post(nds_url, data={"namespace": namespace})
    assert resp.status_code in (200, 201, 409, 422), \
        f"Unexpected response from Data Store during namespace creation: {resp.status_code}"
    print(f"Data Store namespace creation response: {resp}")

create_namespaces(nemo_client=nemo_client, ds_host=NDS_URL, namespace=NMS_NAMESPACE)

Created namespace in Entity Store: xlam-tutorial-ns
Data Store namespace creation response: <Response [201]>


#### Verify Namespaces

The following [Data Store API](https://docs.nvidia.com/nemo/microservices/latest/api/datastore.html) and [Entity Store API](https://docs.nvidia.com/nemo/microservices/latest/api/entity-store.html) list the namespace created in the previous cell.

In [6]:
# Verify Namespace in Data Store (using requests as SDK doesn't cover Data Store)
response = requests.get(f"{NDS_URL}/v1/datastore/namespaces/{NMS_NAMESPACE}")
print(f"Data Store - Status Code: {response.status_code}\nResponse JSON: {response.json()}")

# Verify Namespace in Entity Store
namespace_obj = nemo_client.namespaces.retrieve(namespace_id=NMS_NAMESPACE)
print(f"\nEntity Store - Namespace: {namespace_obj.id}")
print(f"Created at: {namespace_obj.created_at}")
print(f"Description: {namespace_obj.description}")
print(f"Project: {namespace_obj.project}")

Data Store - Status Code: 201
Response JSON: {'namespace': 'xlam-tutorial-ns', 'created_at': '2025-06-20T03:56:39Z', 'updated_at': '2025-06-20T03:56:39Z'}

Entity Store - Namespace: xlam-tutorial-ns
Created at: 2025-06-20 03:56:39.457820
Description: None
Project: None


**Tips**:
To list all available namespaces use
```python
requests.get(f"{NDS_URL}/v1/datastore/namespaces/") # For Data Store
nemo_client.namespaces.list() # For Entity Store
```

To delete a namespace use:
```python
requests.delete(f"{NDS_URL}/v1/datastore/namespaces/{namespace}") # For Data Store
nemo_client.namespaces.delete(namespace) # For Entity Store
```

---
<a id="step-1"></a>
## Step 1: Upload Data to NeMo Data Store

The NeMo Data Store supports data management using the Hugging Face `HfApi` Client. 

**Note that this step does not interact with Hugging Face at all, it just uses the client library to interact with NeMo Data Store.** This is in comparison to the previous notebook, where we used the `load_dataset` API to download the xLAM dataset from Hugging Face's repository.

More information can be found in [documentation](https://docs.nvidia.com/nemo/microservices/latest/manage-entities/tutorials/manage-dataset-files.html#set-up-hugging-face-client-with-nemo-data-store)

### 1.1 Create Repository

In [7]:
repo_id = f"{NMS_NAMESPACE}/{DATASET_NAME}"

In [8]:
from huggingface_hub import HfApi

hf_api = HfApi(endpoint=f"{NDS_URL}/v1/hf", token="")

# Create repo
hf_api.create_repo(
    repo_id=repo_id,
    repo_type='dataset',
)

RepoUrl('datasets/xlam-tutorial-ns/xlam-ft-dataset', endpoint='http://data-store.test/v1/hf', repo_type='dataset', repo_id='xlam-tutorial-ns/xlam-ft-dataset')

Next, creating a dataset programmatically requires two steps: uploading and registration. More information can be found in [documentation](https://docs.nvidia.com/nemo/microservices/latest/manage-entities/datasets/create-dataset.html).

### 1.2 Upload Dataset Files to NeMo Data Store

In [9]:
hf_api.upload_file(path_or_fileobj=train_fp,
    path_in_repo="training/training.jsonl",
    repo_id=repo_id,
    repo_type='dataset',
)

hf_api.upload_file(path_or_fileobj=val_fp,
    path_in_repo="validation/validation.jsonl",
    repo_id=repo_id,
    repo_type='dataset',
)

hf_api.upload_file(path_or_fileobj=test_fp,
    path_in_repo="testing/xlam-test-single.jsonl",
    repo_id=repo_id,
    repo_type='dataset',
)

training.jsonl:   0%|          | 0.00/6.06M [00:00<?, ?B/s]

validation.jsonl:   0%|          | 0.00/1.30M [00:00<?, ?B/s]

xlam-test-single.jsonl:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

CommitInfo(commit_url='', commit_message='Upload testing/xlam-test-single.jsonl with huggingface_hub', commit_description='', oid='e55b3211ba39e6ce80dd8a03a79600183e900f1c', pr_url=None, repo_url=RepoUrl('', endpoint='https://huggingface.co', repo_type='model', repo_id=''), pr_revision=None, pr_num=None)

Other tips:
* Take a look at the `path_in_repo` argument above. If there are more than one files in the subfolders:
    * All the .jsonl files in `training/` will be merged and used for training by customizer.
    * All the .jsonl files in `validation/` will be merged and used for validation by customizer.
* NeMo Data Store generally supports data management using the [HfApi API](https://huggingface.co/docs/huggingface_hub/en/package_reference/hf_api). For example, to delete a repo, you may use - 
```python
   hf_api.delete_repo(
     repo_id=repo_id,
     repo_type="dataset"
)
```

### 1.3 Register the Dataset with NeMo Entity Store

To use a dataset for operations such as evaluations and customizations, register a dataset using the `nemo_client.datasets.create()` method.
Register the dataset to refer to it by its namespace and name afterward.

In [10]:
# Create dataset
dataset = nemo_client.datasets.create(
    name=DATASET_NAME,
    namespace=NMS_NAMESPACE,
    description="Tool calling xLAM dataset in OpenAI ChatCompletions format",
    files_url=f"hf://datasets/{NMS_NAMESPACE}/{DATASET_NAME}",
    project="tool_calling",
)
print(f"Created dataset: {dataset.namespace}/{dataset.name}")
dataset

Created dataset: xlam-tutorial-ns/xlam-ft-dataset


Dataset(files_url='hf://datasets/xlam-tutorial-ns/xlam-ft-dataset', id='dataset-3G75hURMVmSLfNahcqQZd5', created_at=datetime.datetime(2025, 6, 20, 3, 58, 16, 182000), custom_fields={}, description='Tool calling xLAM dataset in OpenAI ChatCompletions format', format=None, hf_endpoint=None, limit=None, name='xlam-ft-dataset', namespace='xlam-tutorial-ns', project='tool_calling', split=None, updated_at=datetime.datetime(2025, 6, 20, 3, 58, 16, 182001))

In [11]:
# Sanity check to validate dataset
dataset_obj = nemo_client.datasets.retrieve(namespace=NMS_NAMESPACE, dataset_name=DATASET_NAME)

print("Files URL:", dataset_obj.files_url)
assert dataset_obj.files_url == f"hf://datasets/{repo_id}"

Files URL: hf://datasets/xlam-tutorial-ns/xlam-ft-dataset


---
<a id="step-2"></a>
## 2. LoRA Customization with NeMo Customizer

### 2.1 Start the Training Job

Start the training job by calling `nemo_client.customization.jobs.create()` method.
The following code sets the training parameters and starts the job.

**The training job will take approximately 45 minutes to complete.**

In [56]:
# Create customization job
# If WANDB_API_KEY is set, we send it in the request header, which will report the training metrics to Weights & Biases (WandB).
if WANDB_API_KEY:
    client_with_wandb = nemo_client.with_options(default_headers={"wandb-api-key": WANDB_API_KEY})
else:
    client_with_wandb = nemo_client

customization = client_with_wandb.customization.jobs.create(
    name="llama-3.2-1b-xlam-ft",
    output_model=CUSTOM_MODEL,
    config=f"{BASE_MODEL}@{BASE_MODEL_VERSION}",
    dataset={"name": DATASET_NAME, "namespace": NMS_NAMESPACE},
    hyperparameters={
        "training_type": "sft",
        "finetuning_type": "lora",
        "epochs": 2,
        "batch_size": 16,
        "learning_rate": 0.0001,
        "lora": {
            "adapter_dim": 32,
            "adapter_dropout": 0.1
        }
    }
)
print(f"Created customization job: {customization.id}")
customization

Created customization job: cust-FarcM8gwhL1XFDXQ57qGLL


CustomizationJobOutput(config='meta/llama-3.2-1b-instruct@v1.0.0+A100', config_snapshot=ConfigSnapshot(base_model='meta/llama-3.2-1b-instruct', max_seq_length=4096, precision='bf16-mixed', training_option=CustomizationTrainingOption(finetuning_type='lora', micro_batch_size=1, num_gpus=1, training_type='sft', data_parallel_size=1, num_nodes=1, pipeline_parallel_size=1, tensor_parallel_size=1, use_sequence_parallel=False), chat_prompt_template=None, name=None, namespace=None, prompt_template='{prompt} {completion}', tokenizer=None), dataset='xlam-tutorial-ns/xlam-ft-dataset', hyperparameters=Hyperparameters(finetuning_type='lora', batch_size=16, distillation=None, epochs=2, learning_rate=0.0001, log_every_n_steps=None, lora=Lora(adapter_dim=32, adapter_dropout=0.1, alpha=16, target_modules=None), p_tuning=None, sequence_packing_enabled=False, sft=None, training_type='sft', val_check_interval=None, weight_decay=None), id='cust-FarcM8gwhL1XFDXQ57qGLL', created_at=datetime.datetime(2025, 6,

**Note**: In the snippet above, the model name and version are passed directly in the `config` argument. However, in production environments, administrators typically create customization **[targets](https://docs.nvidia.com/nemo/microservices/latest/fine-tune/manage-customization-targets/index.html)** and corresponding **[configs](https://docs.nvidia.com/nemo/microservices/latest/fine-tune/manage-customization-configs/index.html)**. This approach allows you to configure once and reuse model configurations for multiple customization jobs. In such cases, you simply reference the created configuration in the `config` argument. For more details, refer to the documentation.

The following code sets variables for storing the job ID and customized model name.

In [6]:
# To track status
JOB_ID = customization.id

customization = nemo_client.customization.jobs.retrieve(JOB_ID)

# This will be the name of the model that will be used to send inference queries to
CUSTOMIZED_MODEL = customization.output_model

**Tips**:
* If you configured the NeMo Customizer microservice with your own [Weights & Biases (WandB)](https://wandb.ai/) API key, you can find the training graphs and logs in your WandB account, "nvidia-nemo-customizer" project. Your run ID is similar to your customization `JOB_ID`.
  
* To cancel a job that you scheduled incorrectly, run the following code.
  
  ```python
  nemo_client.customization.jobs.cancel(job_id=JOB_ID)
  ```

### 2.2 Get Job Status

Get the job status by using the `nemo_client.customization.jobs.status()` method.
The following code sets the job ID and sends the request.

In [20]:
# Get job status
job_status = nemo_client.customization.jobs.status(job_id=JOB_ID)

print("Percentage done:", job_status.percentage_done)
print("Job Status:", json.dumps(job_status.model_dump(), indent=2, default=str))

Percentage done: 100.0
Job Status: {
  "created_at": "2025-06-20 04:20:22.061480",
  "status": "failed",
  "updated_at": "2025-06-20 04:46:50.376819",
  "best_epoch": 2,
  "elapsed_time": 0.0,
  "epochs_completed": 2,
  "metrics": {
    "keys": [
      "train_loss",
      "val_loss"
    ],
    "metrics": {
      "train_loss": [
        {
          "step": 9,
          "timestamp": "2025-06-20T04:32:50.973138",
          "value": 1.8576767444610596
        },
        {
          "step": 19,
          "timestamp": "2025-06-20T04:33:08.341909",
          "value": 0.5704246759414673
        },
        {
          "step": 29,
          "timestamp": "2025-06-20T04:33:26.712158",
          "value": 0.12280075997114182
        },
        {
          "step": 39,
          "timestamp": "2025-06-20T04:33:44.524580",
          "value": 0.04450385272502899
        },
        {
          "step": 49,
          "timestamp": "2025-06-20T04:34:02.972717",
          "value": 0.16312028467655182
        }

In [ ]:
# Add wait job function to wait for the customization job to complete

from time import sleep, time

def wait_job(nemo_client, job_id: str, polling_interval: int = 10, timeout: int = 6000):
    """Helper for waiting an eval job using SDK."""
    start_time = time()
    job = nemo_client.customization.jobs.retrieve(job_id=job_id)
    status = job.status

    while (status in ["pending", "created", "running"]):
        # Check for timeout
        if time() - start_time > timeout:
            raise RuntimeError(f"Took more than {timeout} seconds.")

        # Sleep before polling again
        sleep(polling_interval)

        # Fetch updated status and progress
        job = nemo_client.customization.jobs.retrieve(job_id=job_id)
        status = job.status
        progress = 0.0
        if status == "running" and job.status_details:
            progress = job.status_details.percentage_done or 0.0
        elif status == "completed":
            progress = 100

        print(f"Job status: {status} after {time() - start_time:.2f} seconds. Progress: {progress}%")


    return job

job = wait_job(nemo_client, JOB_ID, polling_interval=5, timeout=2400)

# Wait for 2 minutes, because sometimes, the job is finished, but the finetuned model is not ready in NIM yet.
sleep(120)

**IMPORTANT:** At this point, the customization job should be completed. If waiting for the job to finish failed or the status is not `"completed"`, please check the logs (`job.status_details.status_logs`).

### 2.3 Validate Availability of Custom Model
The following NeMo Entity Store API should display the model when the training job is complete.
The list below shows all models filtered by your namespace and sorted by the latest first.
For more information about this API, see the [NeMo Entity Store API reference](https://docs.nvidia.com/nemo/microservices/latest/api/entity-store.html).
With the following code, you can find all customized models, including the one trained in the previous cells.
Look for the `name` fields in the output, which should match your `CUSTOMIZED_MODEL`.

In [21]:
# List models with filters
models_page = nemo_client.models.list(
    filter={"namespace": NMS_NAMESPACE},
    sort="-created_at"
)

# Print models information
print(f"Found {len(models_page.data)} models in namespace {NMS_NAMESPACE}:")
for model in models_page.data:
    print(f"\nModel: {model.name}")
    print(f"  Namespace: {model.namespace}")
    print(f"  Base Model: {model.base_model}")
    print(f"  Created: {model.created_at}")
    if model.peft:
        print(f"  Fine-tuning Type: {model.peft.finetuning_type}")

Found 1 models in namespace xlam-tutorial-ns:

Model: llama-3.2-1b-xlam-run1@cust-FarcM8gwhL1XFDXQ57qGLL
  Namespace: xlam-tutorial-ns
  Base Model: meta/llama-3.2-1b-instruct
  Created: 2025-06-20 04:20:22.162792
  Fine-tuning Type: lora


 The customized model can also be retrieved directly by using its name.

In [24]:
# CUSTOMIZED_MODEL is constructed as `namespace/model_name`, so we need to extract the model name
model = nemo_client.models.retrieve(namespace=NMS_NAMESPACE, model_name=CUSTOMIZED_MODEL.split("/")[1])

print(f"Model: {model.namespace}/{model.name}")
print(f"Base Model: {model.base_model}")
print(f"Status: {model.artifact.status}")

Model: xlam-tutorial-ns/llama-3.2-1b-xlam-run1@cust-FarcM8gwhL1XFDXQ57qGLL
Base Model: meta/llama-3.2-1b-instruct
Status: upload_completed


NVIDIA NIM directly picks up the LoRA adapters from NeMo Entity Store. You can also query the NIM endpoint to look for it, as shown in the following code.

In [25]:
# Check if the custom LoRA model is hosted by NVIDIA NIM
models = nemo_client.inference.models.list()
model_names = [model.id for model in models.data]

assert CUSTOMIZED_MODEL in model_names, \
    f"Model {CUSTOMIZED_MODEL} not found"

---

<a id="step-3"></a>
## Step 3: Sanity Test the Customized Model By Running Sample Inference

Once the model is customized, its adapter is automatically saved in NeMo Entity Store and is ready to be picked up by NVIDIA NIM.
You can test the model by sending a prompt to its NIM endpoint.

First, choose one of the examples from the test set.

### 3.1 Get Test Data Sample

In [26]:
def read_jsonl(file_path):
    """Reads a JSON Lines file and yields parsed JSON objects"""
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespace
            if not line:
                continue  # Skip empty lines
            try:
                yield json.loads(line)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
                continue


test_data = list(read_jsonl(test_fp))

print(f"There are {len(test_data)} examples in the test set")

There are 713 examples in the test set


In [30]:
# Randomly choose
test_sample = random.choice(test_data)

# Visualize the inputs to the LLM - user query and available tools
test_sample['messages'], test_sample['tools']

([{'role': 'user',
   'content': "Calculate the integral of the function 'x^2 + 3x + 2' from 0 to 10 using the trapezoidal rule."}],
 [{'type': 'function',
   'function': {'name': 'merge_sorted_lists',
    'description': 'Merges two sorted lists into a single sorted list.',
    'parameters': {'type': 'object',
     'properties': {'list1': {'description': 'The first sorted list.',
       'type': 'array'},
      'list2': {'description': 'The second sorted list.', 'type': 'array'}}}}},
  {'type': 'function',
   'function': {'name': 'is_power_of_two',
    'description': 'Checks if a number is a power of two.',
    'parameters': {'type': 'object',
     'properties': {'num': {'description': 'The number to check.',
       'type': 'integer'}}}}},
  {'type': 'function',
   'function': {'name': 'trapezoidal_integration',
    'description': 'Calculates the definite integral of a function using the trapezoidal rule.',
    'parameters': {'type': 'object',
     'properties': {'func': {'description':

### 3.2 Send an Inference Call to NIM

NIM exposes an OpenAI-compatible completions API endpoint, which you can query using the `OpenAI` client library as shown in the following code.

In [28]:
inference_client = OpenAI(
  base_url = f"{NIM_URL}/v1",
  api_key = "None"
)

completion = inference_client.chat.completions.create(
  model = CUSTOMIZED_MODEL,
  messages = test_sample["messages"],
  tools = test_sample["tools"],
  tool_choice = 'auto',
  temperature = 0.1,
  top_p = 0.7,
  max_tokens = 512,
  stream = False
)

completion.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='chatcmpl-tool-1c74f457beee40398662f22c6aaede86', function=Function(arguments='{"a": 56, "b": 98}', name='greatest_common_divisor'), type='function')]

The Python SDK also supports the same inference call, as shown in the following code.

In [31]:
completion = nemo_client.chat.completions.create(
  model = CUSTOMIZED_MODEL,
  messages = test_sample["messages"],
  tools = test_sample["tools"],
  tool_choice = 'auto',
  temperature = 0.1,
  top_p = 0.7,
  max_tokens = 512,
  stream = False
)

completion.choices[0].message.tool_calls

[ChoiceMessageToolCall(id='chatcmpl-tool-c616cace9d8e4693a7aa514ef4c6a31a', function=Function(arguments='{"a": 56, "b": 98}', name='greatest_common_divisor'), type='function')]

Given that the fine-tuning job was successful, you can get an inference result comparable to the ground truth:

In [32]:
# The ground truth answer
test_sample['tool_calls']

[{'type': 'function',
  'function': {'name': 'trapezoidal_integration',
   'arguments': {'func': 'x**2 + 3*x + 2', 'a': 0, 'b': 10}}}]

**Note:** In production environments, application developers typically provide their own set of tools relevant to the specific task. The model must select from these tools based on the given query. To explore this further, you can sample a data point from the dataset to see which tools are available, then experiment by constructing a query and observing the model’s response.

### 3.3 Take Note of Your Custom Model Name

Take note of your custom model name, as you will use it to run evaluations in the subsequent notebook.

In [33]:
print(f"Name of your custom model is: {CUSTOMIZED_MODEL}")

Name of your custom model is: xlam-tutorial-ns/llama-3.2-1b-xlam-run1@cust-FarcM8gwhL1XFDXQ57qGLL
